In [ ]:
%pip install opencv-python

In [ ]:
%pip install mediapipe

In [ ]:
import mediapipe as mp

# Load Model
hands = mp.solutions.hands
hand_landmark = hands.Hands(max_num_hands=1)

In [ ]:
# Camera frame resolution
frame_shape = (1080, 1920, 3)

In [ ]:
# # Import
# import cv2
# import numpy as np

# prevxy = None
# mask = np.zeros(frame_shape, dtype='uint8') # to premanently draw
# colour = (1, 1, 1)
# thickness = 4

# # Access camera
# cap = cv2.VideoCapture(0)
# draw = mp.solutions.drawing_utils

# # Read/Show frame's from camera
# while True:
#     _, frame = cap.read() # BGR -> RGB
#     frame = cv2.flip(frame,1)

    
#     rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     op = hand_landmark.process(rgb)
    
#     if op.multi_hand_landmarks:
#         for all_landmarks in op.multi_hand_landmarks: # 
#             draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)
            
#             x = int(all_landmarks.landmark[8].x*frame_shape[1])
#             y = int(all_landmarks.landmark[8].y*frame_shape[0])
            
#             if prevxy != None:
#                 # draw stuf
#                 cv2.line(mask, prevxy, (x, y), colour, thickness)
#             prevxy = (x, y)
            
#     # Merge Frame and Mask
#     frame = np.where(mask, mask, frame)
    
#     cv2.imshow('Live', frame)
#     if cv2.waitKey(1) == 27: # ESC
#         break

# cap.release()

In [2]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
draw = mp.solutions.drawing_utils

# Initialize camera
cap = cv2.VideoCapture(0)

# Read the first frame to get frame dimensions
_, frame = cap.read()
frame_shape = frame.shape

# Initialize drawing mask and previous coordinates
prevxy = None
mask = np.zeros(frame_shape, dtype='uint8')  # To permanently draw on the mask
colour = (255,255,255)  # White color for drawing
thickness = 9

# Read/Show frames from camera
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally

    # Convert frame to RGB for MediaPipe processing
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hands.process(rgb)

    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks: 
            # Get points for index finger landmarks 5, 6, and 7
            point_A = np.array([all_landmarks.landmark[5].x, all_landmarks.landmark[5].y, all_landmarks.landmark[5].z])  # Point A
            point_B = np.array([all_landmarks.landmark[6].x, all_landmarks.landmark[6].y, all_landmarks.landmark[6].z])  # Point B
            point_C = np.array([all_landmarks.landmark[7].x, all_landmarks.landmark[7].y, all_landmarks.landmark[7].z])  # Point C

            # Calculate vectors
            vector_AB = point_B - point_A
            vector_BC = point_C - point_B

            # Calculate dot product and magnitudes
            dot_product = np.dot(vector_AB, vector_BC)
            magnitude_AB = np.linalg.norm(vector_AB)
            magnitude_BC = np.linalg.norm(vector_BC)

            # Calculate the angle in radians and convert to degrees
            cosine_angle = dot_product / (magnitude_AB * magnitude_BC)
            cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

            angle_degrees = np.degrees(np.arccos(cosine_angle))
            
            # Check if the finger is mostly straight (angle close to 180 degrees)
            draw.draw_landmarks(frame, all_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger (landmark 8)
            x = int(all_landmarks.landmark[8].x * frame_shape[1])
            y = int(all_landmarks.landmark[8].y * frame_shape[0])
            if abs(angle_degrees % 180) > 16:  # You can adjust the threshold as needed

                prevxy = None
                
            else:
                # Reset the previous point if the finger is not straight
                if prevxy != None:
                    cv2.line(mask, prevxy, (x, y), colour, thickness)
                prevxy = (x, y)

    # Merge Frame and Mask
    frame = cv2.add(frame, mask)  # Use cv2.add to merge the mask with the frame
    
    # Show the live frame
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27:  # ESC key
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1725472605.876492 5858579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1725472605.884873 5871896 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725472605.890376 5871894 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
